In [1]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings("default")
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection
import time
from sklearn.neural_network import MLPRegressor

pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000

D:\Repo\Anaconda2\envs\myenv\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Repo\Anaconda2\envs\myenv\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Repo\Anaconda2\envs\myenv\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Repo\Anaconda2\envs\myenv\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Repo\Anaconda2\envs\myenv\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
D:\Repo\Anaconda2\envs\myenv\lib\im

In [13]:
df_merge = pd.read_excel('DATA/prepared/prepared_hard.xlsx')
df_merge_valid = pd.read_excel('DATA/prepared/prepared_hard_valid.xlsx')

In [14]:
df_merge = pd.concat([df_merge, df_merge_valid])

In [15]:
ls_steel = df_merge['марка стали'].value_counts().index
len(ls_steel)

72

In [16]:
col_chem = [
     'C', 
    'Mn', 
    'Si', 
    'P', 
    'S',
    'Cr', 
    'Ni', 
    'Cu', 
    'Al', 
    'V', 
    'Ti', 
    'Nb', 
    'Mo', 
    'N', 
    'B'
]

ls = col_chem + ['марка стали']

In [17]:
def mean_chem(df, steel):
    df = df[df['марка стали'] == steel]
    df[col_chem] = df.groupby(['марка стали'])[col_chem].transform(lambda x: round(x.mean(), 5))
    df = df.reset_index(drop=True)
    df = df.drop_duplicates('C')
    df = df[ls]
    return df

In [18]:
df_merge_1 = df_merge[df_merge['марка стали'] == '37Г2Ф']
df_merge_1[col_chem] = df_merge_1.groupby('марка стали')[col_chem].transform(lambda x: round(x.mean(), 5))
df_merge_1 = df_merge_1.reset_index(drop=True)
df_merge_1 = df_merge_1.drop_duplicates('C')
df_merge_1 = df_merge_1[ls]
df_merge_1

D:\Repo\Anaconda2\envs\myenv\lib\site-packages\pandas\core\frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,B,марка стали
0,0.36533,1.50962,0.282,0.0111,0.00792,0.09713,0.1052,0.14285,0.02908,0.08814,0.00006,0.0001,0.00006,0.00785,NaN,37Г2Ф


In [19]:
df_merge_2 = mean_chem(df_merge, '30Г2')
df_merge_2

D:\Repo\Anaconda2\envs\myenv\lib\site-packages\pandas\core\frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,B,марка стали
0,0.28432,1.33433,0.2583,0.0096,0.00714,0.10216,0.12498,0.19266,0.02702,0.00456,0.00714,0.00553,0.01562,0.00892,0.0005,30Г2


In [20]:
df_tmp = pd.merge(df_merge_1, df_merge_2, how = 'outer')
df_tmp

,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,B,марка стали
0,0.36533,1.50962,0.2820,0.0111,0.00792,0.09713,0.10520,0.14285,0.02908,0.08814,0.00006,0.00010,0.00006,0.00785,NaN,37Г2Ф
1,0.28432,1.33433,0.2583,0.0096,0.00714,0.10216,0.12498,0.19266,0.02702,0.00456,0.00714,0.00553,0.01562,0.00892,0.0005,30Г2


In [21]:
df_merge_steel = df_tmp
for steel in ls_steel: 
    df_merge_steel =  pd.merge(df_merge_steel, mean_chem(df_merge, steel), how = 'outer')

df_merge_steel

D:\Repo\Anaconda2\envs\myenv\lib\site-packages\pandas\core\frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,B,марка стали
0,0.36533,1.50962,0.28200,0.01110,0.00792,0.09713,0.10520,0.14285,0.02908,0.08814,0.00006,0.00010,0.00006,0.00785,NaN,37Г2Ф
1,0.28432,1.33433,0.25830,0.00960,0.00714,0.10216,0.12498,0.19266,0.02702,0.00456,0.00714,0.00553,0.01562,0.00892,0.00050,30Г2
2,0.14293,0.53463,0.27399,0.00827,0.00402,0.55438,0.08593,0.11121,0.03223,0.05289,0.00170,0.00355,0.00700,0.00620,0.00094,13ХФА
3,0.17656,0.57057,0.27012,0.00848,0.00582,0.99551,0.12524,0.17330,0.03268,0.07311,0.01323,0.04006,0.14019,0.00834,NaN,18ХМФБ
4,0.28418,1.43375,0.26280,0.00889,0.00641,0.09727,0.11977,0.18272,0.02649,0.00555,0.00745,0.00722,0.00998,0.00868,NaN,30Г2-2
5,0.38310,1.57095,0.50435,0.01081,0.00776,0.11996,0.13804,0.18235,0.02752,0.00459,0.00006,0.00002,0.00117,0.00870,0.00100,38Г2С
6,0.09218,1.39821,0.59554,0.01026,0.00925,0.07953,0.09887,0.13548,0.03034,0.00567,0.00745,0.00288,0.01199,0.00780,0.00100,09Г2С
7,0.27831,1.35238,0.26280,0.00895,0.00631,0.10958,0.13704,0.19352,0.02993,0.00412,0.00980,0.00493,0.01997,0.01010,NaN,30Г2-8
8,0.29024,1.35896,0.26180,0.00949,0.00654,0.09352,0.11782,0.20908,0.02694,0.00550,0.00900,0.00585,0.00865,0.00901,NaN,30Г2-3
9,0.29327,0.92171,0.26087,0.00930,0.00534,0.68355,0.12665,0.15436,0.03212,0.00114,0.00000,0.00000,0.16058,0.00847,NaN,30ХГМА


In [22]:
df_merge_steel.to_excel('mean_chem_steel.xlsx')

In [23]:
df_merge_steel.shape

(72, 16)